In [1]:
# Install dependencies if needed
!pip install --upgrade scikit-learn xgboost lightgbm shap

# Import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import shap

# Sklearn modules
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, PowerTransformer

# XGBoost & LightGBM
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from google.colab import drive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.4
    Uninstalling xgboost-2.1.4:
      Successfully uninstalled xgboost-2.1.4
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.5.0
    Uninstalling lightgbm-4.5.0:
      Successfully uninstalled lightgbm-4.5.0
  Attempting uninstall: shap
    Found existing installation: shap 0.46.0
    Uninstalling shap-0.46.0:
      Successfully uninstalled shap-0.46.0


In [2]:
# Define file path
file_url = "https://raw.githubusercontent.com/svvd-m/real-estate-price-prediction/main/DATA2.csv"
df = pd.read_csv(file_url)

print(" Dataset loaded successfully!")
display(df.head(10))

# Basic Data Overview
print("\n Data Info:")
df.info()

print("\n Missing Values:")
display(df.isnull().sum())


 Dataset loaded successfully!


,Sl no,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Distance,...,Bathroom,Car,Landsize,BuildingArea,YearBuilt,CouncilArea,Lattitude,Longtitude,Regionname,Propertycount
0,1,Abbotsford,85 Turner St,2,h,1480000,S,Biggin,03-12-16,2.5,...,1.0,1.0,202.0,NaN,NaN,Yarra,-37.7996,144.9984,Northern Metropolitan,4019.0
1,2,Abbotsford,25 Bloomburg St,2,h,1035000,S,Biggin,04-02-16,2.5,...,1.0,0.0,156.0,79.0,1900.0,Yarra,-37.8079,144.9934,Northern Metropolitan,4019.0
2,4,Abbotsford,5 Charles St,3,h,1465000,SP,Biggin,04-03-17,2.5,...,2.0,0.0,134.0,150.0,1900.0,Yarra,-37.8093,144.9944,Northern Metropolitan,4019.0
3,5,Abbotsford,40 Federation La,3,h,850000,PI,Biggin,04-03-17,2.5,...,2.0,1.0,94.0,NaN,NaN,Yarra,-37.7969,144.9969,Northern Metropolitan,4019.0
4,6,Abbotsford,55a Park St,4,h,1600000,VB,Nelson,04-06-16,2.5,...,1.0,2.0,120.0,142.0,2014.0,Yarra,-37.8072,144.9941,Northern Metropolitan,4019.0
5,10,Abbotsford,129 Charles St,2,h,941000,S,Jellis,07-05-16,2.5,...,1.0,0.0,181.0,NaN,NaN,Yarra,-37.8041,144.9953,Northern Metropolitan,4019.0
6,11,Abbotsford,124 Yarra St,3,h,1876000,S,Nelson,07-05-16,2.5,...,2.0,0.0,245.0,210.0,1910.0,Yarra,-37.8024,144.9993,Northern Metropolitan,4019.0
7,14,Abbotsford,98 Charles St,2,h,1636000,S,Nelson,08-10-16,2.5,...,1.0,2.0,256.0,107.0,1890.0,Yarra,-37.8060,144.9954,Northern Metropolitan,4019.0
8,15,Abbotsford,217 Langridge St,3,h,1000000,S,Jellis,08-10-16,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0
9,16,Abbotsford,18a Mollison St,2,t,745000,S,Jellis,08-10-16,2.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Northern Metropolitan,4019.0



 Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18396 entries, 0 to 18395
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Sl no          18396 non-null  int64  
 1   Suburb         18396 non-null  object 
 2   Address        18396 non-null  object 
 3   Rooms          18396 non-null  int64  
 4   Type           18396 non-null  object 
 5   Price          18396 non-null  int64  
 6   Method         18396 non-null  object 
 7   SellerG        18396 non-null  object 
 8   Date           18396 non-null  object 
 9   Distance       18395 non-null  float64
 10  Postcode       18395 non-null  float64
 11  Bedroom2       14927 non-null  float64
 12  Bathroom       14925 non-null  float64
 13  Car            14820 non-null  float64
 14  Landsize       13603 non-null  float64
 15  BuildingArea   7762 non-null   float64
 16  YearBuilt      8958 non-null   float64
 17  CouncilArea    12233 non-null  object

,0
Sl no,0
Suburb,0
Address,0
Rooms,0
Type,0
Price,0
Method,0
SellerG,0
Date,0
Distance,1


In [ ]:
# Check the distribution of 'Price'
print(f" Initial Skewness: {df['Price'].skew():.3f}, Kurtosis: {df['Price'].kurtosis():.3f}")

# Visualize raw price distribution
plt.figure(figsize=(7,4))
sns.histplot(df['Price'], bins=50, kde=True)
plt.title("Original Price Distribution")
plt.show()


In [ ]:
from scipy.stats import boxcox

# Create copies to avoid modifying original data
df['Log_Price'] = np.log1p(df['Price'])  # Log transformation
df['SQRT_Price'] = np.sqrt(df['Price'])  # Square root transformation

# Box-Cox transformation (requires strictly positive values)
df['BoxCox_Price'], _ = boxcox(df['Price'] + 1)  # Adding 1 to avoid zero-values issue

# Yeo-Johnson (works with zero & negative values)
yj_transformer = PowerTransformer(method='yeo-johnson')
df['YeoJohnson_Price'] = yj_transformer.fit_transform(df[['Price']])

# Check skewness & kurtosis after transformation
print("\n Skewness & Kurtosis After Transformations:")
print(f"Price: Skewness={df['Price'].skew():.3f}, Kurtosis={df['Price'].kurtosis():.3f}")
print(f"Log_Price: Skewness={df['Log_Price'].skew():.3f}, Kurtosis={df['Log_Price'].kurtosis():.3f}")
print(f"SQRT_Price: Skewness={df['SQRT_Price'].skew():.3f}, Kurtosis={df['SQRT_Price'].kurtosis():.3f}")
print(f"BoxCox_Price: Skewness={df['BoxCox_Price'].skew():.3f}, Kurtosis={df['BoxCox_Price'].kurtosis():.3f}")
print(f"YeoJohnson_Price: Skewness={df['YeoJohnson_Price'].skew():.3f}, Kurtosis={df['YeoJohnson_Price'].kurtosis():.3f}")

# Compare distributions visually
fig, axes = plt.subplots(2, 2, figsize=(12,10))

sns.histplot(df['Log_Price'], bins=50, kde=True, ax=axes[0,0])
axes[0,0].set_title("Log_Price Distribution")

sns.histplot(df['SQRT_Price'], bins=50, kde=True, ax=axes[0,1])
axes[0,1].set_title("SQRT_Price Distribution")

sns.histplot(df['BoxCox_Price'], bins=50, kde=True, ax=axes[1,0])
axes[1,0].set_title("BoxCox_Price Distribution")

sns.histplot(df['YeoJohnson_Price'], bins=50, kde=True, ax=axes[1,1])
axes[1,1].set_title("YeoJohnson_Price Distribution")

plt.show()


In [ ]:
# Decision: Yeo-Johnson produced the most normal distribution
df['Price'] = df['YeoJohnson_Price']  # Replace original price column

# Drop temporary columns
df.drop(columns=['Log_Price', 'SQRT_Price', 'BoxCox_Price', 'YeoJohnson_Price'], inplace=True)

print("\n Selected Yeo-Johnson Transformation for Normalization.")


In [ ]:
# Drop unnecessary columns
cols_to_drop = ['Sl no', 'Address', 'Method', 'SellerG', 'Date', 'Postcode', 'Bedroom2']
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True, errors='ignore')

# Fill missing numeric values with median
num_cols = ['Distance', 'Bathroom', 'Car', 'Landsize', 'BuildingArea', 'YearBuilt',
            'Lattitude', 'Longtitude', 'Propertycount', 'YearSold', 'MonthSold', 'YearMonthSold']

for col in num_cols:
    if col in df.columns:
        df[col].fillna(df[col].median(), inplace=True)

# Fill missing categorical values with mode
cat_cols = ['CouncilArea', 'Regionname', 'Type', 'Suburb']
for col in cat_cols:
    if col in df.columns:
        df[col].fillna(df[col].mode()[0], inplace=True)

print("\n Data Preprocessing Completed.")


In [ ]:
# Separate target and features
y = df['Price']
X = df.drop('Price', axis=1)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f" Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# One-Hot Encode Categorical Features
cat_cols = ['Type', 'Suburb', 'CouncilArea', 'Regionname']
X_train = pd.get_dummies(X_train, columns=cat_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=cat_cols, drop_first=True)

# Align train and test data
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(" Encoding & Scaling Complete.")


In [ ]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score

# Define models to test
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "XGBoost": XGBRegressor(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=300, learning_rate=0.1, max_depth=7, random_state=42)
}

# Function to evaluate models
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)  # Train model
    train_score = model.score(X_train, y_train)  # R² on Train
    test_score = model.score(X_test, y_test)  # R² on Test
    mae = mean_absolute_error(y_test, model.predict(X_test))  # MAE
    rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test)))  # RMSE
    return train_score, test_score, mae, rmse

# Store results
model_results = {}

# Iterate through models & evaluate
for name, model in models.items():
    train_r2, test_r2, mae, rmse = evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test)
    model_results[name] = [train_r2, test_r2, mae, rmse]
    print(f" {name}: Train R²: {train_r2:.3f}, Test R²: {test_r2:.3f}, MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# Convert results to DataFrame
model_comparison_df = pd.DataFrame.from_dict(model_results, orient='index', columns=['Train R²', 'Test R²', 'MAE', 'RMSE'])

# Display Model Comparison
print("\n Model Performance Comparison:")
display(model_comparison_df)


In [ ]:
import shap

# Initialize SHAP for XGBoost
explainer_xgb = shap.TreeExplainer(models["XGBoost"])
shap_values_xgb = explainer_xgb.shap_values(X_test_scaled)

# SHAP Summary Plot
shap.summary_plot(shap_values_xgb, X_test_scaled, feature_names=X_train.columns)

# Initialize SHAP for LightGBM
explainer_lgbm = shap.TreeExplainer(models["LightGBM"])
shap_values_lgbm = explainer_lgbm.shap_values(X_test_scaled)

# SHAP Summary Plot
shap.summary_plot(shap_values_lgbm, X_test_scaled, feature_names=X_train.columns)


In [ ]:
from sklearn.inspection import permutation_importance

# Permutation Importance for XGBoost
perm_importance_xgb = permutation_importance(models["XGBoost"], X_test_scaled, y_test, scoring="r2", n_repeats=5, random_state=42)
xgb_feature_importance = pd.DataFrame({"Feature": X_train.columns, "Importance": perm_importance_xgb.importances_mean})
xgb_feature_importance = xgb_feature_importance.sort_values(by="Importance", ascending=False)

print(" XGBoost Permutation Importance:")
display(xgb_feature_importance.head(10))

# Permutation Importance for LightGBM
perm_importance_lgbm = permutation_importance(models["LightGBM"], X_test_scaled, y_test, scoring="r2", n_repeats=5, random_state=42)
lgbm_feature_importance = pd.DataFrame({"Feature": X_train.columns, "Importance": perm_importance_lgbm.importances_mean})
lgbm_feature_importance = lgbm_feature_importance.sort_values(by="Importance", ascending=False)

print(" LightGBM Permutation Importance:")
display(lgbm_feature_importance.head(10))


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define parameter grid for XGBoost
xgb_param_grid = {
    "n_estimators": [300, 500],
    "max_depth": [6, 8],
    "learning_rate": [0.1, 0.05],
    "colsample_bytree": [0.8, 1.0],
    "subsample": [0.8, 1.0]
}

# Perform Grid Search for XGBoost
xgb_grid_search = GridSearchCV(XGBRegressor(random_state=42), param_grid=xgb_param_grid, scoring="r2", cv=3, verbose=1, n_jobs=-1)
xgb_grid_search.fit(X_train_scaled, y_train)

print(" Best XGBoost Parameters:", xgb_grid_search.best_params_)

# Define parameter grid for LightGBM
lgbm_param_grid = {
    "n_estimators": [300, 500],
    "max_depth": [6, 8],
    "learning_rate": [0.1, 0.05],
    "colsample_bytree": [0.8, 1.0],
    "subsample": [0.8, 1.0]
}

# Perform Grid Search for LightGBM
lgbm_grid_search = GridSearchCV(LGBMRegressor(random_state=42), param_grid=lgbm_param_grid, scoring="r2", cv=3, verbose=1, n_jobs=-1)
lgbm_grid_search.fit(X_train_scaled, y_train)

print(" Best LightGBM Parameters:", lgbm_grid_search.best_params_)


In [ ]:
# Train tuned XGBoost
xgb_tuned = XGBRegressor(**xgb_grid_search.best_params_, random_state=42)
xgb_tuned.fit(X_train_scaled, y_train)

# Train tuned LightGBM
lgbm_tuned = LGBMRegressor(**lgbm_grid_search.best_params_, random_state=42)
lgbm_tuned.fit(X_train_scaled, y_train)

# Predict
y_pred_xgb_tuned = xgb_tuned.predict(X_test_scaled)
y_pred_lgbm_tuned = lgbm_tuned.predict(X_test_scaled)

# Evaluate performance
mae_xgb_tuned = mean_absolute_error(y_test, y_pred_xgb_tuned)
r2_xgb_tuned = r2_score(y_test, y_pred_xgb_tuned)

mae_lgbm_tuned = mean_absolute_error(y_test, y_pred_lgbm_tuned)
r2_lgbm_tuned = r2_score(y_test, y_pred_lgbm_tuned)

# Compare models
comparison_df = pd.DataFrame({
    "Model": ["XGBoost", "Tuned XGBoost", "LightGBM", "Tuned LightGBM"],
    "MAE": [model_results["XGBoost"][2], mae_xgb_tuned, model_results["LightGBM"][2], mae_lgbm_tuned],
    "R² Score": [model_results["XGBoost"][1], r2_xgb_tuned, model_results["LightGBM"][1], r2_lgbm_tuned]
})

print("\n Final Model Comparison:")
display(comparison_df)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Residual Plot for XGBoost
residuals_xgb = y_test - y_pred_xgb_tuned
plt.figure(figsize=(7,5))
sns.histplot(residuals_xgb, bins=50, kde=True)
plt.title("Residual Distribution - XGBoost")
plt.show()

# Residual Plot for LightGBM
residuals_lgbm = y_test - y_pred_lgbm_tuned
plt.figure(figsize=(7,5))
sns.histplot(residuals_lgbm, bins=50, kde=True)
plt.title("Residual Distribution - LightGBM")
plt.show()


In [ ]:
# SHAP for tuned XGBoost
explainer_xgb_tuned = shap.TreeExplainer(xgb_tuned)
shap_values_xgb_tuned = explainer_xgb_tuned.shap_values(X_test_scaled)

shap.summary_plot(shap_values_xgb_tuned, X_test_scaled, feature_names=X_train.columns)

# SHAP for tuned LightGBM
explainer_lgbm_tuned = shap.TreeExplainer(lgbm_tuned)
shap_values_lgbm_tuned = explainer_lgbm_tuned.shap_values(X_test_scaled)

shap.summary_plot(shap_values_lgbm_tuned, X_test_scaled, feature_names=X_train.columns)


In [ ]:
# Train XGBoost Model
xgb_model = XGBRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    colsample_bytree=0.8,
    subsample=0.8,
    random_state=42
)

xgb_model.fit(X_train_scaled, y_train)

# Train LightGBM Model
lgb_model = LGBMRegressor(
    n_estimators=500,
    max_depth=8,
    learning_rate=0.05,
    colsample_bytree=0.8,
    subsample=0.8,
    random_state=42
)

lgb_model.fit(X_train_scaled, y_train)

print(" Models trained successfully.")


In [ ]:
if 'xgb_model' not in globals():
    raise ValueError(" ERROR: XGBoost model is not trained! Run the training section first.")

if 'lgb_model' not in globals():
    raise ValueError(" ERROR: LightGBM model is not trained! Run the training section first.")


In [ ]:
from sklearn.inspection import permutation_importance

# Compute feature importance for XGBoost
xgb_importance = permutation_importance(xgb_model, X_test_scaled, y_test, n_repeats=5, random_state=42)
xgb_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': xgb_importance.importances_mean})
xgb_feature_importance = xgb_feature_importance.sort_values(by='Importance', ascending=False)

# Compute feature importance for LightGBM
lgb_importance = permutation_importance(lgb_model, X_test_scaled, y_test, n_repeats=5, random_state=42)
lgb_feature_importance = pd.DataFrame({'Feature': X_train.columns, 'Importance': lgb_importance.importances_mean})
lgb_feature_importance = lgb_feature_importance.sort_values(by='Importance', ascending=False)

# Display top features
print("\n XGBoost Feature Importance:")
display(xgb_feature_importance.head(10))

print("\n LightGBM Feature Importance:")
display(lgb_feature_importance.head(10))


In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Compute VIF for each feature
vif_data = pd.DataFrame()
vif_data["Feature"] = X_train.columns
vif_data["VIF"] = [variance_inflation_factor(X_train_scaled, i) for i in range(X_train_scaled.shape[1])]

# Display high VIF values (>5 indicates potential multicollinearity)
print("\n VIF Analysis (Features with High Multicollinearity):")
display(vif_data[vif_data["VIF"] > 5])


In [ ]:
# Identify low-importance and high-VIF features to remove
low_importance_features = xgb_feature_importance[xgb_feature_importance['Importance'] < 0.005]['Feature'].tolist()
high_vif_features = vif_data[vif_data["VIF"] > 5]['Feature'].tolist()

# Combine both sets and drop them from training/testing data
features_to_drop = list(set(low_importance_features + high_vif_features))
X_train_selected = X_train.drop(columns=features_to_drop)
X_test_selected = X_test.drop(columns=features_to_drop)

print(f"\n Removed {len(features_to_drop)} low-impact or highly correlated features:")
print(features_to_drop)

# Scale the updated feature set
scaler = StandardScaler()
X_train_selected_scaled = scaler.fit_transform(X_train_selected)
X_test_selected_scaled = scaler.transform(X_test_selected)

print("\n Feature selection and scaling complete.")
